In [ ]:
from rethon.ensemble_generation import EnsembleGenerator, SimpleEnsembleGenerator
from rethon.ensemble_generation import create_random_argument_list, standard_model_params_varied_alphas
from rethon.ensemble_generation import random_position_as_set, create_random_arguments2
from rethon.model import StandardPosition, DAGDialecticalStructure, StandardGlobalReflectiveEquilibrium
from os import getcwd
from multiprocessing import Process
import time

In [ ]:
def experiment(options):
    arguments_list = options[0]
    n = options [1]
    coms_list = options [2]
    implementations = options [3]
    filename = options [4]
    
    start = time.perf_counter()

    model_params = [{
        'neighbourhood_depth': 1,
        'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0],
        'account_penalties': [0.0, 0.3, 1.0, 1.0],
        'weights': {'account': 0.35, 'faithfulness': 0.1, 'systematicity': 0.55}
    }]
    ensemble_gen = SimpleEnsembleGenerator(arguments_list = arguments_list, 
                                     n_sentence_pool = n,
                                     initial_commitments_list = coms_list,
                                     model_parameters_list = model_params,
                                     implementations = implementations)


    # removing items we don't need
    ensemble_gen.remove_item('account_penalties')
    ensemble_gen.remove_item('faithfulness_penalties')
    ensemble_gen.remove_item('weight_account')
    ensemble_gen.remove_item('weight_systematicity')
    ensemble_gen.remove_item('weight_faithfulness')
    ensemble_gen.remove_item('n_random_choices')
    ensemble_gen.remove_item('n_init_coms_min_ax_base')
    ensemble_gen.remove_item('n_fixed_point_coms_min_ax_base')
    ensemble_gen.remove_item('fixed_point_coms_min_ax_bases')
    ensemble_gen.remove_item('init_coms_min_ax_bases')

    # defining the function for the global Optima
    def data_global_optima(ensemble_generator):
        re_model = ensemble_generator.reflective_equilibrium()
        #if re_model.model_name() == "StandardModel": fix for new version....
        if re_model.model_name() == "GlobalNumpyReflectiveEquilibrium":
            return re_model.global_optima(ensemble_generator.initial_commitments())

    ensemble_gen.add_item('global optima', data_global_optima)
    argument_size = len(arguments_list[0])
    #max_premises = 

    ensemble_gen.ensemble_items_to_csv(
                                  output_file_name = f'data_output_{filename}.csv',
                                  output_dir_name = getcwd(),
                                  archive = False, # save the csv as archived tar.gz
                                  save_preliminary_results = True, # will create preliminary csv-data sets 
                                  preliminary_results_interval = 50)
    finish = time.perf_counter()
    print(f'{filename}: {round(finish-start,2)}')
    return f'{filename}: {round(finish-start,2)}'

In [ ]:
def createStruct(argument_size, max_premises, pool):
    # Sentence pool
    n= pool
    # Two random dialectical structures with 7 arguments each
    n_dialectical_structures = 20
    arguments_list = [create_random_arguments2(n_sentences=n, 
                                               n_arguments=argument_size,
                                               n_max_premises=max_premises) for i in range(n_dialectical_structures)]
    # Two random initial commitments
    n_positions = 2
    coms_list = [random_position_as_set(n_sentences = n) for i in range(n_positions)]
    local_impl = [{'module_name': 'rethon.model',
               'position_class_name':'StandardPosition',
               'dialectical_structure_class_name': 'BDDDialecticalStructure',
               'reflective_equilibrium_class_name': 'StandardLocalReflectiveEquilibrium'
               }]
    global_impl = [{'module_name': 'rethon.model',
                'position_class_name':'StandardPosition',
                'dialectical_structure_class_name': 'BDDDialecticalStructure',
                'reflective_equilibrium_class_name': 'StandardGlobalReflectiveEquilibrium'}]
    return [[arguments_list, n, coms_list, local_impl, f"args-{argument_size}_premises-{max_premises}_pool-{n}_model-local"],[arguments_list, n, coms_list, global_impl, f"args-{argument_size}_premises-{max_premises}_pool-{n}_model-global"]]
            

args-7_premises-2_pool-5_model-local: 9.97
args-7_premises-2_pool-5_model-global: 10.48
args-7_premises-2_pool-6_model-local: 25.07
args-7_premises-2_pool-6_model-global: 27.31
args-7_premises-2_pool-7_model-global: 86.99
args-7_premises-2_pool-7_model-local: 96.81
args-7_premises-2_pool-8_model-global: 223.34
args-7_premises-2_pool-8_model-local: 333.25
args-7_premises-2_pool-9_model-global: 1117.32
args-7_premises-2_pool-9_model-local: 1449.93
args-7_premises-2_pool-10_model-global: 3779.62
args-7_premises-2_pool-10_model-local: 5655.59
args-7_premises-2_pool-11_model-global: 7325.45
args-7_premises-2_pool-11_model-local: 18764.94

BDD:
args-7_premises-2_pool-8_model-global: 648.03
args-7_premises-2_pool-8_model-local: 671.24
args-7_premises-2_pool-9_model-global: 2275.82
args-7_premises-2_pool-9_model-local: 2479.25


In [ ]:
#print(experiment(createStruct(7,2,9)[0]))
#with concurrent.futures.ThreadPoolExecutor() as executor:
#    counter = []
#    for argsize in range(20):
#        counter.append(executor.map(experiment, createStruct(7,2,argsize+5)))

#    for count in counter:
#        for s in count:
#            print(s)
#options = createStruct(2,1)[0]
#experiment(options)

if __name__ == '__main__':
    for argsize in range(20):
        print('new round')
        options = createStruct(7,2,argsize+5)
        p1 = Process(target=experiment, args=(options[0],))
        p2 = Process(target=experiment, args=(options[1],))
        p1.start()
        p2.start()
        if argsize % 4 == 0:
            p1.join()
            p2.join()

new round
args-7_premises-2_pool-5_model-local: 4.39
args-7_premises-2_pool-5_model-global: 7.11
new round
new round
new round
new round
args-7_premises-2_pool-6_model-local: 20.53
args-7_premises-2_pool-6_model-global: 27.16
args-7_premises-2_pool-7_model-local: 64.7
args-7_premises-2_pool-7_model-global: 80.93
args-7_premises-2_pool-8_model-local: 220.38
args-7_premises-2_pool-8_model-global: 281.46
args-7_premises-2_pool-9_model-global: 743.79
args-7_premises-2_pool-9_model-local: 868.99
new round
new round
new round
new round
args-7_premises-2_pool-10_model-global: 3112.25
args-7_premises-2_pool-10_model-local: 4773.51
args-7_premises-2_pool-11_model-global: 14401.48
args-7_premises-2_pool-11_model-local: 22881.2
